<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">IRIS Flower Classification using TensorFlow</p>


![image.png](https://miro.medium.com/v2/resize:fit:1200/1*S2GII1ul0JjbZ0YaFvDByw.jpeg)

<p style="color:  #8a4baf; font-size: 20px; padding: 20px; border-radius: 10px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">Iris veri kümesi, makine öğrenimi ve istatistik alanında iyi bilinen ve sıklıkla kullanılan bir ölçüttür. Üç farklı iris çiçeği türünün çeşitli özelliklerinin ölçümlerinin bir koleksiyonunu içeren klasik bir veri kümesidir: Setosa, Versicolor ve Virginica. 
Bu veri kümesi, veri analizi, sınıflandırma ve örüntü tanıma tekniklerini keşfetmek ve uygulamak için temel bir örnek olarak hizmet vermektedir. Basitliği ve anlaşılırlığı ile Iris veri seti, hem yeni başlayanlar hem de deneyimli uygulayıcılar için bir mihenk taşı olmaya devam etmekte ve veri keşfi ve algoritma değerlendirmesi hakkında içgörüler sağlamaktadır.</p>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam 

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report

In [ ]:
df = pd.read_csv("iris.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Feature'lar üzerinden istatistiksel bilgilere bakalım

df.describe()       

In [ ]:
# Hedef değişkenimiz üzerinden istatistiksel bilgilere bakalım

df.groupby("species").agg(["min","max","std","mean"]) 

In [ ]:
# Hedef değişkenimizin dağılımı

plt.figure(figsize=(8, 6))
ax=sns.countplot(data=df, x='species')
ax.bar_label(ax.containers[0])
plt.title('Species Distribution in Iris Dataset')
plt.xlabel('Species')
plt.ylabel('Count')
plt.show()

![](https://data-flair.training/blogs/wp-content/uploads/sites/2/2021/10/iris-flower.webp)

In [ ]:
# Aykırı değerlerin durumu

sns.set(style = "ticks")
plt.figure(figsize = (12,10))

plt.subplot(2,2,1)                                                 
sns.boxplot(x = "species", y = "sepal_length", data = df)
plt.subplot(2,2,2)
sns.boxplot(x = "species", y = "sepal_width", data = df)
plt.subplot(2,2,3)
sns.boxplot(x = "species", y = "petal_length", data = df)
plt.subplot(2,2,4)
sns.boxplot(x = "species", y = "petal_width", data = df)
plt.show()

In [ ]:
sns.set(style="whitegrid")

# Plot the distributions of features for each species
sns.pairplot(df, hue='species', markers=["o", "s", "D"], diag_kind="kde")
plt.show()

In [ ]:
plt.figure(figsize = (10,11))
sns.heatmap(df.corr(), annot = True, cmap = "coolwarm")
plt.show()

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(18, 6))

# Petal width
plt.subplot(1, 4, 1)
sns.histplot(data=df, x="petal_width", hue="species", stat="density", kde=True)
plt.title("Petal Width")

# Petal length
plt.subplot(1, 4, 2)
sns.histplot(data=df, x="petal_length", hue="species", stat="density", kde=True)
plt.title("Petal Length")

# Sepal length
plt.subplot(1, 4, 3)
sns.histplot(data=df, x="sepal_length", hue="species", stat="density", kde=True)
plt.title("Sepal Length")

# Sepal width
plt.subplot(1, 4, 4)
sns.histplot(data=df, x="sepal_width", hue="species", stat="density", kde=True)
plt.title("Sepal Width")

plt.tight_layout()
plt.show()

### Sonuçlar

* Dört sayısal veri barındıran sütunumuz varken, analiz etmeyi hedeflediğimiz veri olan sadece bir kategorik sütunumuz vardır (çiçek türleri).

* Veri kümesi dengelidir, yani her üç tür için eşit kayıtlar mevcuttur.

* Verimiz de hiç eksik değer yok.

* Boxplot grafiği üzeinden gördüğümüz kadarıyla ciddi bir Outlier sorunumuz yok.

* Taç yaprak genişliği ile taç yaprak uzunluğu arasında güçlü bir korelasyon mevcuttur.

* Setosa türleri, küçük boyutlu olmasından dolayı dolayı en kolay ayırt edilebilen türdür.

* Versicolor ve Virginica türleri genellikle karıştırılır ve bazen ayrılması zordur. Ancak genellikle Versicolor türünün boyutları daha ortalama değerdedir. Buna karşın virginica türünün boyutları daha büyüktür.

In [ ]:
# target değişkenimizi numeric forma dönüştürüyoruz

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit_transform(df["species"])

In [ ]:
df["species"] = le.fit_transform(df["species"])

In [ ]:
df["species"].value_counts()

In [ ]:
# target değişkeninin nümeric dönüşüm sonrası feature'lar ile korelasyonu 

df.corr()['species'][:-1].sort_values().plot.barh();

In [ ]:
df.head()

### Train-Test Split

In [ ]:
X = df .drop("species", axis = 1)
y = df["species"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## ANN Model

#### ÖNEMLİ:
* Veri setinizdeki hedef değişkenler sınıf numaraları şeklinde ise, yani her sınıf bir sayı ile temsil ediliyorsa, **sparse_categorical_crossentropy** kayıp fonksiyonunu kullanmanız daha uygun olacaktır. Eğer one-hot kodlaması kullanılıyorsa, o zaman **categorical_crossentropy** tercih edilir. Biz taget değişkenimizin durumundan dolayı sparse_categorical_crossentropy loss fonksiyonunu tercih edicez 
* Eğer modelimizde categorical_crossentropy kullanıcak olsaydık önce **from tensorflow.keras.utils import to_categorical** import edip sonra da y verimiz üzerinde sırayla **y_train_encoded = to_categorical(y_train)** ve **y_test_encoded = to_categorical(y_test)** yapmamız gerekecekti

In [ ]:
# Build the model
model = Sequential([
    Dense(units=10, activation="relu"),
    Dense(units=10, activation="relu"),             # nöron sayısını azalttığımıızda epoch arttırmak gerekti
    Dense(units=3, activation="softmax")
])

opt = Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(x=X_train, y=y_train, 
                    validation_data=(X_test, y_test),
                    batch_size=5, epochs=200)

In [ ]:
model.summary()

* Verimiz büyük ve kompleks bir form da olmadığı için Dropout ve Early stopping yöntemlerini uygulamadık

In [ ]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred_probabilities = model.predict(X_train)
    y_train_pred = y_train_pred_probabilities.argmax(axis=1)
    y_pred_probabilities = model.predict(X_test)
    y_pred = y_pred_probabilities.argmax(axis=1)
    
    print("Test Set:")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    print("\nTrain Set:")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

In [ ]:
eval_metric(model, X_train, y_train, X_test, y_test)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,6))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
# Sadece Accuracy değişimine bakalım

print("\nAccuracy during Training :\n------------------------------------")

pd.DataFrame(history.history)["accuracy"].plot(figsize=(10, 5))
plt.title("Accuracy improvements with Epoch")
plt.show()

## Final Model and Model Deployment

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
scaler = MinMaxScaler().fit(X)

In [ ]:
# MinMaxScaler işleminin kaydedilmesi
pickle.dump(scaler, open("scaler_iris", 'wb'))

In [ ]:
X_scaled = scaler.transform(X)

In [ ]:
final_model = Sequential([
            Dense(units=10, activation="relu"),
            Dense(units=10, activation="relu"),             # nöron sayısını azalttığımıızda epoch arttırmak gerekti
            Dense(units=3, activation="softmax")
    ])

opt = Adam(learning_rate=0.001)

final_model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

final_model.fit(x=X_scaled, y=y, batch_size=5, epochs=200) 

In [ ]:
# Modelin kaydedilmesi

final_model.save('final_model_ANN_iris.h5') 

### Kaydedilmiş Model Üzerinden Prediction 

In [ ]:
# Manuel datamızı oluşturuyoruz 

my_dict = {"sepal_length": [4.5, 5.8, 7.5],
           "sepal_width": [2.2, 3, 4.2],
           "petal_length": [1.3, 4.3, 6.5],
           "petal_width": [0.2, 1.3, 2.3]}

sample_ann = pd.DataFrame(my_dict)
sample_ann

In [ ]:
# min-max scaler dosyasının yüklenmesi
scaler_iris = pickle.load(open("scaler_iris", "rb"))

In [ ]:
sample_scaled = scaler_iris.transform(sample_ann)
sample_scaled

In [ ]:
# daha önceden eğitilmiş modelin yüklenmsi

from tensorflow.keras.models import load_model

In [ ]:
final_model = load_model('final_model_ANN_iris.h5')

In [ ]:
final_model.predict(sample_scaled)

In [ ]:
y_pred_probabilities = final_model.predict(sample_scaled)
y_pred = y_pred_probabilities.argmax(axis=1)

In [ ]:
sample_ann["pred"] = y_pred
sample_ann

In [ ]:
predictions = final_model.predict(sample_scaled)

normalized_predictions = tf.nn.softmax(predictions, axis=-1).numpy()

In [ ]:
normalized_predictions

In [ ]:
sample_ann["pred_proba_setosa"] = normalized_predictions[:,0]
sample_ann["pred_proba_versicolor"] = normalized_predictions[:,1]
sample_ann["pred_proba_virginica"] = normalized_predictions[:,2]
sample_ann

## xgboost Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X_train.shape

In [ ]:
import xgboost as xgb
xgb_cls = xgb.XGBClassifier(objective = "multiclass:softmax", num_class = 3)

In [ ]:
xgb_cls.fit(X_train,y_train)

In [ ]:
preds = xgb_cls.predict(X_test)

In [ ]:
preds

In [ ]:
np.array(y_test)

In [ ]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

In [ ]:
eval_metric(xgb_cls, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_validate

model = xgb.XGBClassifier(objective = "multiclass:softmax", num_class = 3)

scores = cross_validate(model, X_train, y_train, scoring = ['accuracy', 'precision_weighted','recall_weighted',
                                                            'f1_weighted'], cv = 20)
                                                        
df_scores = pd.DataFrame(scores, index = range(1, 21))
df_scores.mean()[2:]

## Final Model and Model Deployment

In [ ]:
final_model = xgb.XGBClassifier(objective = "multiclass:softmax", num_class = 3).fit(X, y)

In [ ]:
pickle.dump(final_model, open("final_model_xgboost_iris", 'wb'))

### Kaydedilmiş Model Üzerinden Prdiction 

In [ ]:
my_dict = {"sepal_length": [4.5, 5.8, 7.5],
           "sepal_width": [2.2, 3, 4.2],
           "petal_length": [1.3, 4.3, 6.5],
           "petal_width": [0.2, 1.3, 2.3]}
sample_xgboost = pd.DataFrame(my_dict)
sample_xgboost

In [ ]:
final_model = pickle.load(open("final_model_xgboost_iris", "rb"))

In [ ]:
predictions = final_model.predict(sample_xgboost)
predictions_proba = final_model.predict_proba(sample_xgboost)

In [ ]:
predictions_proba

In [ ]:
sample_xgboost["pred"] = predictions
sample_xgboost["pred_proba_setosa"] = predictions_proba[:,0]
sample_xgboost["pred_proba_versicolor"] = predictions_proba[:,1]
sample_xgboost["pred_proba_virginica"] = predictions_proba[:,2]
sample_xgboost

In [ ]:
sample_ann         # yukarıda XGBoost tahminleri ile aşağıda ANN tahminlerini karşılaştırabilriz

## Alternatif Denemeler

* Bu aşamada loss function olarak sparse_categorical_crossentropy yerine categorical_crossentropy deneyip çeşitli parametre ayarlamalarıyla sonuçlarımızı karşılaştırmaya çalıştık

### categorical_crossentropy kullanımı

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
iris_data = load_iris() # load the iris dataset
x = iris_data.data
y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column
# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
#print(y)
# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)
# Build the model
model = Sequential()
model.add(Dense(10, input_shape=(4,), activation='relu', name='fc1'))
model.add(Dense(10, activation='relu', name='fc2'))
model.add(Dense(3, activation='softmax', name='output'))
# Adam optimizer with learning rate of 0.001
optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
print('Neural Network Model Summary: ')
print(model.summary())
# Train the model
model.fit(train_x, train_y, verbose=2, batch_size=5, epochs=200)
# Test on unseen data
results = model.evaluate(test_x, test_y)
print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

In [ ]:
model.summary()

* Verimiz büyük ve kompleks bir form da olmadığı için Dropout ve Early stopping yöntemlerini uygulamadık

In [ ]:
def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred_probabilities = model.predict(X_train)
    y_train_pred = y_train_pred_probabilities.argmax(axis=1)
    y_pred_probabilities = model.predict(X_test)
    y_pred = y_pred_probabilities.argmax(axis=1)
    
    print("Test Set:")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    print("\nTrain Set:")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

In [ ]:
eval_metric(model, X_train, y_train, X_test, y_test)

In [ ]:
# Manuel datamızı oluşturuyoruz 

my_dict = {"sepal_length": [4.5, 5.8, 7.5],
           "sepal_width": [2.2, 3, 4.2],
           "petal_length": [1.3, 4.3, 6.5],
           "petal_width": [0.2, 1.3, 2.3]}

sample_ann = pd.DataFrame(my_dict)
sample_scaled = scaler.transform(sample_ann)

In [ ]:
sample_scaled

In [ ]:
predictions = model.predict(sample_scaled)

normalized_predictions = tf.nn.softmax(predictions, axis=-1).numpy()

In [ ]:
normalized_predictions

### Hyperparameter Tuning deneme sonuçları

* 3 hidden 16,8,4 - batch size 5 - accuracy 0.70 - epoch 200
* 3 hidden 16,8,4 - batch size 30 - accuracy 0.96 -  epoch 200
* 2 hidden 16,8  - batch size 30 - accuracy 0.92 -  epoch 200
* 2 hidden 16,8  - batch size 20 - accuracy 0.96 - epoch 200
* 2 hidden 16,8  - batch size 20 - accuracy 0.96 - epoch 100
* 2 hidden 8,4  - batch size 5 - accuracy 0.96  - epoch 100 -  learning_rate=0.001
* 2 hidden 8,4  - batch size 5 - accuracy 1.0 -  epoch 100 - learning_rate=0.003 
* 2 hidden 8,4  - batch size 10 - accuracy 0.96  - epoch 100 - learning_rate=0.003 - epoch 150 aynı sonuç
* 2 hidden 8,4  - batch size 5 - accuracy 1.0 -  epoch 150  - learning_rate=0.003  -  test size 0.1   -  loss 0,0336     **tercih 1
* 2 hidden 8,4  - batch size 20 - accuracy 1.0  - epoch 150 - learning_rate=0.003 
* 2 hidden 8,4  - batch size 30 - accuracy 0.96 - epoch 150 - learning_rate=0.003 
* 2 hidden 8,4  - batch size 40 - accuracy 0.96 -  epoch 200 - learning_rate=0.003
* 2 hidden 64,64  - batch size 14 - accuracy 0.96 -  epoch 20 - learning_rate=0.003 
* 2 hidden 64,64  - batch size 10 - accuracy 0.93  - epoch 30 - learning_rate=0.003 
* 2 hidden 64,64  - batch size 10 - accuracy 0.93 -  epoch 30 - learning_rate=0.008  - loss 0,0545
* 2 hidden 128,128  - batch size 10 - accuracy 0.96 - epoch 30 - learning_rate=0.008  - loss 0,0460
* 2 hidden 128,128  - batch size 20 - accuracy 0.96  - epoch 50 - learning_rate=0.008  - loss 0,0428
* 2 hidden 128,128  - batch size 30 - accuracy 1.0 - epoch 70 - learning_rate=0.008  - loss 0,0336
* 2 hidden 128,128  - batch size 30 - accuracy 1.0 -  epoch 80 - learning_rate=0.01  - loss 0,0412
* 2 hidden 128,128  - batch size 40 - accuracy 1.0  - epoch 80 - learning_rate=0.01  - loss 0,0412
* 2 hidden 128,128  - batch size 14 - accuracy 0.96 - epoch 20 - learning_rate=0.003  

## Sonuç ve Çıkarımlar

* Genel olarak 2 hidden layer seçeneklerimiz de daha yüksek score lar elde ettik
* Nöron sayılarını belirlerken iki seçenek karşımıza çıktı 
  - Eğer düşük nöronlar(16, 8, 4 gibi) seçtiysek, 5-40 arası batch size için 100-200 epoch denemeleri ile 0.96 ile 1.00 arası score'lar elde ettik
  - Eğer yüksek nöronlar(64, 128 gibi) seçtiysek, 10-40 arası batch size için 20-100 epoch denemeleri ile 0.96 ile 1.00 arası score'lar elde ettik
* Burdan çıkardığımız sonuçlar:
    - Yüksek nöron seçiminde düşük epoch denemelerinin yeterli olduğu
    - Düşük batch size seçiminde yüksek epoch denemelerinin gerekmediği (burada ki kıyaslama yüksek batch size seçimlerine göre)
    - Verinin az olduğu durumlarda ANN de çok fazla altenatif parametre ayarları yapabilsekde sonucun çok değişmediği görüldü. Hatta XGBoost modeline göre çiçek türü tahminlerinde ANN düşük probability ile karar verdiği görüldü. Örneğin aşağıda ki 2 modelin tahminlerinde XGBoost yüksek probability(0.99) ile çiçek türüne karar verirken, ANN de bir çok farklı parametre denemelerine rağmen max 0.58 ile çiçek türü tahminlerini gerçekleştirdiği görüldü.

In [ ]:
sample_xgboost     # xgboost modeli tahmin sonuçları 

In [ ]:
sample_ann       # ANN modeli tahmin sonuçları        

# Streamlit 

### EDA - Prediction (XGBOOST - ANN)

In [ ]:
%%writefile iris_app.py
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pickle
from keras import models
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

iris = pd.read_csv("iris.csv")

st.markdown(
    """
    <style>
    .stApp {
        background-color: #e6dd27;  /* Koyu sarı renk */
    </style>
    """,
    unsafe_allow_html=True
)


# Sekmeleri ekleyin
selected_tab = st.sidebar.selectbox("Select Page", ["EDA", "Prediction"])

if selected_tab == "EDA":
   
    #st.sidebar.header('User Input Parameters for EDA')
    # EDA ile ilgili kodları burada ekleyin, veri görselleştirmeleri vb.
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">Exploring Data Analysis (EDA)</p>', unsafe_allow_html=True)
    st.image("https://data-flair.training/blogs/wp-content/uploads/sites/2/2021/10/iris-flower.webp", use_column_width=True)
    
    plt.figure(figsize=(8, 6))
    ax=sns.countplot(data=iris, x='species')
    ax.bar_label(ax.containers[0])
    plt.title('Species Distribution in Iris Dataset')
    plt.xlabel('Species')
    plt.ylabel('Count')
    st.pyplot(plt)
    
    
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">Statistical Properties of the Features</p>', unsafe_allow_html=True)
    st.dataframe(iris.describe())
    
    
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">Plot the Distributions of Features For Each Species</p>', unsafe_allow_html=True)
    sns.set(style="whitegrid")
    sns.pairplot(iris, hue='species', markers=["o", "s", "D"], diag_kind="kde")
    st.pyplot(plt)
    
    
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">Status of Outliers</p>', unsafe_allow_html=True)
    sns.set(style = "ticks")
    plt.figure(figsize = (12,10))

    plt.subplot(2,2,1)                                                 
    sns.boxplot(x = "species", y = "sepal_length", data = iris)
    plt.subplot(2,2,2)
    sns.boxplot(x = "species", y = "sepal_width", data = iris)
    plt.subplot(2,2,3)
    sns.boxplot(x = "species", y = "petal_length", data = iris)
    plt.subplot(2,2,4)
    sns.boxplot(x = "species", y = "petal_width", data = iris)
    st.pyplot(plt)
    
    
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">Correlation Table</p>', unsafe_allow_html=True)
    plt.figure(figsize = (10,11))
    sns.heatmap(iris.corr(), annot = True, cmap = "coolwarm")
    st.pyplot(plt)
    
elif selected_tab == "Prediction":
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);">IRIS Flower Classification using TensorFlow</p>', unsafe_allow_html=True)
    st.image("https://miro.medium.com/v2/resize:fit:1200/1*S2GII1ul0JjbZ0YaFvDByw.jpeg", use_column_width=True)

    # Metni ortala
    st.markdown('<p style="text-align: center; font-size: 20px;">This app predicts the <strong>Iris flower</strong> type!</p>', unsafe_allow_html=True)
     
    model_option = st.sidebar.selectbox("Select Model", ["XGBoost", "Artificial Neural Network"])
    # Tahmin sekmesi içeriği
    st.sidebar.header('User Input Parameters for Prediction')
    
    def user_input_features():
        sepal_length = st.sidebar.slider('Sepal length', 4.3, 7.9, 5.4, step=0.1)
        sepal_width = st.sidebar.slider('Sepal width', 2.0, 4.4, 3.4)
        petal_length = st.sidebar.slider('Petal length', 1.0, 6.9, 1.3)
        petal_width = st.sidebar.slider('Petal width', 0.1, 2.5, 0.2)
        data = {'sepal_length': sepal_length,
                'sepal_width': sepal_width,
                'petal_length': petal_length,
                'petal_width': petal_width}
        features = pd.DataFrame(data, index=[0])
        return features
    
    df = user_input_features()

    # Load local Iris dataset
    #iris = pd.read_csv("iris.csv")

    final_xgboost = pickle.load(open("final_model_xgboost_iris", "rb"))
    final_ann = load_model('final_model_ANN_iris.h5')
    scaler_iris = pickle.load(open("scaler_iris", "rb"))

    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">User Input parameters</p>', unsafe_allow_html=True)
    st.dataframe(df.style.format("{:.2f}"))


    if model_option == "XGBoost":
        prediction = final_xgboost.predict(df)
        prediction_proba = final_xgboost.predict_proba(df)
        
    elif model_option == "Artificial Neural Network":     
        df_scaled = scaler_iris.transform(df)
        y_pred_probabilities = final_ann.predict(df.values)
        prediction = y_pred_probabilities.argmax(axis=1)
        prediction_proba = tf.nn.softmax(y_pred_probabilities, axis=1).numpy()

    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">Class labels and their corresponding index number</p>', unsafe_allow_html=True)
    st.write(iris.species.value_counts().index)
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">Prediction Probability</p>', unsafe_allow_html=True)
    st.write(prediction_proba)
    st.markdown('<p style="background-color: #8a4baf; color: white; font-size: 20px; padding: 10px; border-radius: 5px; text-align: left; box-shadow: 0px 2px 3px rgba(0, 0, 0, 0.1);">Prediction</p>', unsafe_allow_html=True)

    # İki sütunlu düzen oluşturma
    col1, col2 = st.columns([2, 1])  # İlk sütunun genişliği 2, ikincisinin genişliği 1

    # Sol sütun: Yazı
    with col1:
        st.write("<p style='color:red; font-size:25px; text-align: center; margin-top: 150px;'>Prediction: {}</p>".format(", ".join(iris.species.value_counts().index[prediction])), unsafe_allow_html=True)

    # Sağ sütun: GIF
    with col2:
        st.markdown('<div style="display: flex; justify-content: flex-end;"><img src="https://media.tenor.com/VmR4yzFeI_AAAAAi/floral-fury-cuphead.gif" alt="GIF" width="100%" style="max-width: 200px; margin-right: 150px;"></div>', unsafe_allow_html=True)


In [ ]:
!streamlit run iris_app.py